In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

Loaded environment variables from .env file.


In [2]:
from datasets import load_dataset
from research_tools.utils import set_seed

set_seed(42)

# Parameters
num_samples = 10000  # Number of samples to collect

# Load the dataset in streaming mode
dataset = load_dataset("HuggingFaceFW/fineweb-edu", split="train", streaming=True)
shuffled_dataset = dataset.shuffle(seed=42, buffer_size=10_000)
# Reservoir sampling
# Define the number of samples you want
num_samples = 10000

# Use an iterator to avoid downloading everything
sampled_data = []
for idx, item in enumerate(dataset):
    if idx < num_samples:
        sampled_data.append(item)
    else:
        break

print(f"Collected {len(sampled_data)} samples.")

Resolving data files:   0%|          | 0/1630 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1630 [00:00<?, ?it/s]

Collected 10000 samples.


In [6]:
from pathlib import Path
import json

data_dir = Path("../data")

file_path = data_dir / "fineweb-edu" / "corpus_split_0.jsonl"
# split data into 5
for i in range(5):
    start = num_samples // 5 * i
    end = num_samples // 5 * (i + 1)
    split = sampled_data[start:end]
    file_path = data_dir / "fineweb-edu" / f"corpus_split_{i}.jsonl"
    file_path.parent.mkdir(parents=True, exist_ok=True)
    with open(file_path, "w") as f:
        for item in split:
            f.write(json.dumps(item))
            f.write("\n")